# Déployez un modèle dans le cloud

In [1]:
#check spark home is set
import os
print(os.environ['SPARK_HOME'])
print(os.environ['JAVA_HOME'])
import findspark 
findspark.init()

C:\spark\spark-3.2.0-bin-hadoop3.2
C:\Program Files\Java\jdk1.8.0_311


In [2]:
import pandas as pd
import numpy as np
import base64
import scipy.sparse

In [3]:
#imports spark con, session
import pyspark
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession, types
from pyspark.sql.types import ArrayType, FloatType, IntegerType, StructType, StructField, StringType
from pyspark.sql.functions import udf, regexp_replace, array, col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.image import ImageSchema

from pyspark.ml.linalg import DenseVector, VectorUDT, Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT
import pyspark.sql.functions as funcs

In [4]:
#import sikitimage
from skimage import io
from skimage.feature import  ORB
from skimage.color import rgb2gray


In [34]:


def extract_orb(image_path, descriptor_extractor):  
    image = io.imread(image_path)
    image_gs = rgb2gray(image) 
    descriptors =[]
    descriptors_ =[]
    try :     
        descriptor_extractor = ORB(n_keypoints = 200)
        descriptor_extractor.detect_and_extract(image_gs)
        descriptors = descriptor_extractor.descriptors*1
        descriptors = [int(i) for i in descriptors.flatten()]
        descriptors_ = [int(i) for i in descriptors]
        descriptors_ = [sum(col) / float(len(col)) for col in zip(*descriptors)]
    except:
        pass 
    return descriptors, descriptors_

def get_repos(rootdir):
    repositories =[]
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            repositories.append(d)
    print(repositories)
    return repositories
    
def load_image_data(link_path , image_df):
    image_data = image_df.filter(image_df.origin == link_path).select("image.data")
    print(image_data)
    return image_data

def create_df_from_repo(repo_list, index, list_input, list_):
    descriptor_extractor = ORB()
    r = repositories[index]
    image_df = spark.read.format("image").load(r).select("image.origin", "image.data")
    category = r[r.rfind('/')+1:]
    for f in os.listdir(os.path.abspath(r)):
        path= r + '/' +os.path.basename(f)
        val, val_ = extract_orb(path, descriptor_extractor)
        if len(val)> 0:
            element = [path , category, val]
            list_.append(val_) 
            list_input.append(element)  
    return image_df, list_input, list_

def rdd_to_df(list_input, sc):
    rdd1 = sc.parallelize(list_input)
    type(rdd1)
    rdd1.collect()
    #convert the  RDD to a dataframe with columns : link_path, category, loaded_image, ord_feature, reduced_orb
    schema = StructType([
        StructField('origin', StringType(), True),
        StructField('category', StringType(), True),
        StructField('orb_feature', ArrayType(IntegerType()), True)
    ])
    df = spark.createDataFrame(rdd1,schema)
    type(df)
    df.limit(5).show()    
    return df


In [22]:
#udfs 
udf_load_image = udf(lambda img: load_image_data(link_path , image_df), ArrayType(FloatType()))
udf_trans_type = udf(lambda vs: Vectors.dense(vs), VectorUDT())

In [7]:
#sc.stop()
conf = pyspark.SparkConf().setAppName('fruitApp').setMaster('local')
sc  = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [35]:
#create a list 
list_input = []
list_ =[]

In [36]:
rootdir = r"../data/subset/"
repositories = get_repos(rootdir)

#initialize image_df and list for the first folder
image_df, list_input, list_ = create_df_from_repo(repositories, 0, list_input, list_)
for i in range(1, len(repositories))  :
    image_df_i , list_input, list_ = create_df_from_repo(repositories, i, list_input, list_)
    image_df = image_df.union(image_df_i)
    

print(len(list_input))

['../data/subset/Apricot', '../data/subset/Clementine']
614


In [37]:
df = rdd_to_df(list_input, sc)

+--------------------+--------+--------------------+
|              origin|category|         orb_feature|
+--------------------+--------+--------------------+
|../data/subset/Ap...| Apricot|[1, 1, 0, 1, 1, 1...|
|../data/subset/Ap...| Apricot|[1, 1, 0, 1, 1, 1...|
|../data/subset/Ap...| Apricot|[0, 1, 0, 1, 1, 1...|
|../data/subset/Ap...| Apricot|[0, 1, 0, 1, 1, 1...|
|../data/subset/Ap...| Apricot|[1, 1, 0, 1, 1, 1...|
+--------------------+--------+--------------------+



In [38]:
#transform type

df = df.withColumn("orb_feature", udf_trans_type(df.orb_feature))

In [40]:
def merge_df(df1, df2):
    df1 = df1.join(other=df2, on=['origin'], how='left')
    df1.limit(5).show()
    return df1

In [ ]:
image_df = image_df.withColumn('origin', col("origin").cast("String"))
image_df = image_df.withColumn('origin', regexp_replace('origin', 'file:///', ''))

df = merge_df(df, image_df)

In [ ]:
df.limit(5).show()

In [ ]:

assembler = VectorAssembler(inputCols=df.columns[2:3], outputCol='features')
df2 = assembler.transform(df)
df2.limit(3).show()

In [ ]:
#SVD dimensional reduction
def extractSVD(rdd):
    mat = RowMatrix(rdd)

    # Compute the top 5 singular values and corresponding singular vectors.
    svd = mat.computeSVD(5, computeU=True)
    U = svd.U       # The U factor is a RowMatrix.
    s = svd.s       # The singular values are stored in a local dense vector.
    V = svd.V       # The V factor is a local dense matrix.
    return U,s,V
U,s,V = 

In [ ]:


#apply pca
pca = PCA(k=4, inputCol="features", outputCol="reduced_orb")
model = pca.fit(df2.select('features'))
pca_feat_df = model.transform(df2)

In [39]:
# functions 
print(list_[0])
def pca_feature (list_):
    
    rdd = sc.parallelize(list_)
    mat = RowMatrix(rdd)
    # Compute the top 4 principal components.
    # Principal components are stored in a local dense matrix.
    pc = mat.computePrincipalComponents(4)

    # Project the rows to the linear space spanned by the top 4 principal components.
    projected = mat.multiply(pc)
    print(projected)
    return projected 
projected = pca_feature (list_)

[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]


Py4JJavaError: An error occurred while calling o413.computePrincipalComponents.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 20) (DESKTOP-T2U7JQD executor driver): java.lang.IllegalArgumentException: requirement failed: Dimensions mismatch when adding new sample. Expecting 256 but got 1280.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.stat.SummarizerBuffer.add(Summarizer.scala:495)
	at org.apache.spark.mllib.stat.Statistics$.$anonfun$colStats$1(Statistics.scala:59)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:194)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.fold

In [ ]:
from keras.applications import InceptionV3
from pyspark.ml.feature import PCA
from sparkdl import DeepImageFeaturizer, DeepImagePredictor
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sparkdl.image.image import ImageSchema
from sparkdl import DeepImagePredictor

featurizer = DeepImageFeaturizer(
    inputCol="image",
    outputCol="features",
    modelName="Resnet50"
)
model = InceptionV3(weights="imagenet")
model.save('/tmp/model-full.h5')
def loadAndPreprocessKerasInceptionV3(uri):
    # this is a typical way to load and prep images in keras
    image = img_to_array(load_img(uri, target_size=(299, 299)))
    image = np.expand_dims(image, axis=0)
    return preprocess_input(image)

transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile="/tmp/model-full.h5",
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

In [ ]:
predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels",
                               modelName="InceptionV3", decodePredictions=True, topK=10)
image_df = ImageSchema.readImages("/data/myimages")
predictions_df = predictor.transform(image_df)

# Partie AWS

In [ ]:

#Instalation


In [ ]:
env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt', # path relative to `source_dir` below.
}
sagemaker_model = TensorFlowModel(model_data = 's3://mybucket/modelTarFile,
                                  role = role,
                                  entry_point = 'entry.py',
                                  code_location = 's3://mybucket/runtime-code/',
                                  source_dir = 'src',
                                  env = env,
                                  name = 'model_name',
                                  sagemaker_session = sagemaker_session,
                                 )

## Création d’une instance SageMaker.

## Création de notebook à l’intérieur de l’instance 
utiliser le kernel conda3. charger les librairies nécessaires.


## Paramétrage pour S3 
pour utiliser S3, définir certains objets :  le nom du compartiment de S3, le chemin vers les données à partir de S3, la région et la session via la fonction boto3, l’initialisation de s3_client et s3.

In [ ]:
import config.p8_fruits_credentials as credentials

PREFIX = 'data'
bucket='p8-sample-bucket'
data_path = 's3a://{}/{}'.format(bucket, PREFIX)

# Get the flickr_key and flickr_secret
ACCESS_KEY = credentials.ACCESS_KEY
SECRET_KEY = credentials.SECRET_KEY
import boto3

s3_resource = boto3.resource(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='eu-west-3',
)

my_bucket = s3_resource.Bucket('p8-fruits')

## Stockage de l'output sous format parquet

In [ ]:
image_df
  .write
  .format("parquet")
  .mode("overwrite")
  .option("compression", "gzip")
  .save("s3://p8-sample-bucket/output/images.parquet")

In [ ]:

!pip install sparkdl --upgrade
!pip show sparkdl
# Build our model

featurizer = DeepImageFeaturizer(
    inputCol="image",
    outputCol="features",
    modelName="ResNet50"
)
features_df = featurizer.transform(image_df)
features_df.printSchema()

In [ ]:
#If you have just one dense vector this will do it:

def dense_to_sparse(vector):
    return _convert_to_vector(scipy.sparse.csc_matrix(vector.toArray()).T)

dense_to_sparse(densevector)
to_sparse = udf(dense_to_sparse, VectorUDT())
DF.withColumn("sparse", to_sparse(col("densevector"))

In [ ]:
from sparkdl import DeepImageFeaturizer
feat = DeepImageFeaturizer(inputCol="image",
                           outputCol="image_features",
                           modelName="ResNet50")

## PCA on the extracted features

from pyspark.ml.feature import PCA
pca = PCA(k=8,
          inputCol="image_features",
          outputCol="pca_features")

## Put feature extractor and PCA in a pipeline

from pyspark.ml import Pipeline
pipe = Pipeline(stages=[feat, pca])
extractor = pipe.fit(images_df)
pca_feat_df = extractor.transform(images_df)